In [1]:
#set up environment 

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#dependency creation 
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 5s (57.6 kB/s)
Reading package li

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-17 22:43:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.31MB/s    in 0.2s    

2022-12-17 22:43:59 (5.31 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
#establish a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
#load Pet Products file 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [6]:
#check df data types 
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

### Create DataFrames to match tables

In [7]:
#import sql functions for pyspark 
from pyspark.sql.functions import to_date
from pyspark.sql.functions import sum,avg,max,count

# Create a review dataset as a DataFrame to prep for removing duplicates
dupe_review_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

#confirm
dupe_review_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| REAKC26P07MDN|   28794885|B00Q0K9604|     510387886| 2015-08-31|
|R3NU7OMZ4HQIEG|   11488901|B00MBW5O9W|     912374672| 2015-08-31|
|R14QJW3XF8QO1P|   43214993|B0084OHUIO|     902215727| 2015-08-31|
|R2HB7AX0394ZGY|   12835065|B001GS71K2|     568880110| 2015-08-31|
| RGKMPDQGSAHR3|   26334022|B004ABH1LG|     692846826| 2015-08-31|
|R1DJCVPQGCV66E|   22283621|B00AX0LFM4|     590674141| 2015-08-31|
|R3V52EAWLPBFQG|   14469895|B00DQFZGZ0|     688538603| 2015-08-31|
|R3DKO8J1J28QBI|   50896354|B00DIRF9US|     742358789| 2015-08-31|
| R764DBXGRNECG|   18440567|B00JRCBFUG|     869798483| 2015-08-31|
| RW1853GAT0Z9F|   50502362|B000L3XYZ4|     501118658| 2015-08-31|
|R33GITXNUF1AD4|   33930128|B00BOEXWFG|     454737777| 2015-08-31|
|R1H7AVM81TAYRV|   43534290|B001HBBQKY|     420905252| 2015-08

In [8]:
#remove duplicates from dataframe
review_df = dupe_review_df.dropDuplicates(["review_id"])

#confirm
review_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100065P6TTS3J|   42711834|B000HHM6PS|      64793786| 2014-04-06|
|R10007MH6NTVFM|   25105396|B0090Z9FFC|     675354291| 2015-06-24|
|R1000CIZTRNP23|   25423435|B00K1B6RCI|     308737701| 2015-03-06|
|R1000JOVLD0J41|   34245087|B000TZ7022|     870738517| 2010-11-13|
|R1000YCFR347I2|   48610961|B00CO527IC|     935823254| 2014-09-05|
|R1000ZYX4P32OD|   27895609|B002DXCQMM|     184150150| 2015-02-27|
|R10015VI9KRLUW|   18810197|B005IZOB5M|     259423137| 2014-03-27|
|R1001KC9GI6ARJ|   11232277|B0080YGXPE|     551772997| 2015-04-16|
|R1001RSHOZYRB6|   25116815|B003TWKJQQ|     188103873| 2014-08-14|
|R1001T0JNBRBS9|    6219052|B005CO91TK|      56263948| 2014-10-01|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [9]:
# Create the customers_table DataFrame and prep to remove duplicates
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
dup_customers_df = df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
#confirm
dup_customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
+-----------+--------------+
only showing top 10 rows



In [10]:
#eliminate duplicate customer ID records
customers_df = dup_customers_df.dropDuplicates(["customer_id"])

#confirm
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
+-----------+--------------+
only showing top 10 rows



In [11]:
# Create the products_table DataFrame and prepare to drop duplicates. 
# products_df = df.select([]).drop_duplicates()
dupe_products_df = df.select(["product_id", "product_title"])

#confirm
dupe_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Q0K9604|(8-Pack) EZwhelp ...|
|B00MBW5O9W|Warren Eckstein's...|
|B0084OHUIO|Tyson's True Chew...|
|B001GS71K2|Soft Side Pet Cra...|
|B004ABH1LG|EliteField 3-Door...|
|B00AX0LFM4|Carlson 68-Inch W...|
|B00DQFZGZ0|Dog Seat Cover Wi...|
|B00DIRF9US|The Bird Catcher ...|
|B00JRCBFUG|Cat Bed - Purrfec...|
|B000L3XYZ4|PetSafe Drinkwell...|
|B00BOEXWFG|Contech ZenDog Ca...|
|B001HBBQKY|Wellness Crunchy ...|
|B007O1FHB0|Rx Vitamins Essen...|
|B001P3NU30|Virbac C.E.T. Enz...|
|B00ZP6HS6S|Kitty Shack - 2 i...|
|B00IP05CUA|Wellness Kittles ...|
|B001U8Y598|OmniPet Anti-Ant ...|
|B011AY4JWO|K9KONNECTION [New...|
|B00DDSHE5A|SUNSEED COMPANY 3...|
|B00PJW5OR8|CXB1983(TM)Cute P...|
+----------+--------------------+
only showing top 20 rows



In [12]:
#remove duplicates 
products_df = dupe_products_df.dropDuplicates(["product_id"])

#confirm
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0310824230|Advantage Flea Co...|
|0615553605|Pet Qwerks Treat ...|
|0684836483|250 Things You Ca...|
|0763004227|Golden Retriever ...|
|0764102885|Saint Bernards (B...|
|0764118269|  The Finch Handbook|
|0764144294|             Gerbils|
|0876051468|The Stray Cat Han...|
|0972585419|Feathered Phonics...|
|0975412817|The Notes Pocket ...|
+----------+--------------------+
only showing top 10 rows



In [13]:
# Create the vine_table. DataFrame and prepare to remove duplicates 
# vine_df = df.select([])

dupe_vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])

#confirm
dupe_vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [14]:
#remove duplicates
type_vine_df = dupe_vine_df.dropDuplicates(["review_id"])

#confirm
type_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100065P6TTS3J|          4|            0|          0|   N|                Y|
|R10007MH6NTVFM|          3|            0|          0|   N|                Y|
|R1000CIZTRNP23|          4|            3|          3|   N|                Y|
|R1000JOVLD0J41|          2|            0|          0|   N|                N|
|R1000YCFR347I2|          3|            1|          1|   N|                Y|
|R1000ZYX4P32OD|          5|            0|          0|   N|                Y|
|R10015VI9KRLUW|          5|            0|          0|   N|                Y|
|R1001KC9GI6ARJ|          4|            1|          1|   N|                Y|
|R1001RSHOZYRB6|          3|            0|          0|   N|                Y|
|R1001T0JNBRBS9|          4|            0|          0|   N|     

In [16]:
#import dependency to allow casting to Integer 
from pyspark.sql.types import IntegerType

#cast star_rating field from string to Integer 
vine_df = type_vine_df.withColumn("star_rating", type_vine_df["star_rating"].cast(IntegerType()))

#confirm 
vine_df.show(10)

#confirm data types 
vine_df.dtypes


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100065P6TTS3J|          4|            0|          0|   N|                Y|
|R10007MH6NTVFM|          3|            0|          0|   N|                Y|
|R1000CIZTRNP23|          4|            3|          3|   N|                Y|
|R1000JOVLD0J41|          2|            0|          0|   N|                N|
|R1000YCFR347I2|          3|            1|          1|   N|                Y|
|R1000ZYX4P32OD|          5|            0|          0|   N|                Y|
|R10015VI9KRLUW|          5|            0|          0|   N|                Y|
|R1001KC9GI6ARJ|          4|            1|          1|   N|                Y|
|R1001RSHOZYRB6|          3|            0|          0|   N|                Y|
|R1001T0JNBRBS9|          4|            0|          0|   N|     

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [17]:
# Configure settings for RDS
# set up and Store environmental variables
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://vine.cs7xws4wbv4h.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [18]:
# Write customers_df to table in vine db

customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [19]:
# Write products_df to table in vine db

products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [20]:
# Write review_df to table in vine db
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [21]:
# Write vine_df to table in vine db

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)